In [493]:
import os
import pandas as pd
import pyautogui
import urllib.parse
import time
import pyperclip
import quopri
from bs4 import BeautifulSoup
from email import policy
from email.parser import BytesParser
import requests
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import numpy as np
import pickle

# 1.Data Collection

#### (1). Download LinkedIn Search Results Webpages

In [2]:
# Create an automatic process to download the job search pages in Linkedin
def auto_download_search(url,file_name):
    pyautogui.hotkey('ctrl', '2') # switch Google Chrome tabs
    time.sleep(1)
    
    pyautogui.click(207,55) # click the browser's address bar.
    time.sleep(0.5)
    
    pyautogui.hotkey('ctrl', 'a') # select all content in the address bar.
    pyperclip.copy(url) # copy the url
    time.sleep(0.5)
    
    pyautogui.hotkey('ctrl', 'v') # paste the url
    time.sleep(0.5)
    
    pyautogui.hotkey('enter') # access the url
    time.sleep(10)
    
    pyautogui.click(882,1023)
    pyautogui.mouseDown() # scroll down the page to look through all job posts
    time.sleep(10)
    pyautogui.mouseUp()# release the left mouse button
    
    pyautogui.hotkey('ctrl', 's') # save the page to local path
    time.sleep(2)
    pyperclip.copy(file_name) # copy the file name
    pyautogui.hotkey('ctrl', 'v') # paste the file name
    time.sleep(2)
    pyautogui.hotkey('enter') # save the page with given name

In [3]:
# create a process to look through all the pages of search results
def search_page_download():
    for i in range(0,1000,25):
        url = f"https://www.linkedin.com/jobs/search/?currentJobId=3733476715&f_SB2=1&f_TPR=r2592000&geoId=103644278&keywords=data%20scientist&location=United%20States&origin=JOB_SEARCH_PAGE_JOB_FILTER&refresh=true&sortBy=R&spellCorrectionEnabled=true&start={i}"
        file_name = f"data_science_job_{int(i/25)}.mhtml"
        auto_download_search(url, file_name)

In [4]:
# download all pages to collect 1000 job posts related to data science
search_page_download()

(2). Extract Individual Job Post URLs from LinkedIn Search Results

In [410]:
# extract urls of job posts from all the pages
def job_url(file_path):
    urls = []
    # Open the MHTML file in binary mode and parse it
    with open(file_path, 'rb') as file:
        msg = BytesParser(policy=policy.default).parse(file)

    # Decode the HTML part correctly
    html_part = None
    for part in msg.walk():
        content_type = part.get_content_type()
        if content_type == 'text/html':
            html_part = part.get_payload(decode=True)
            break
    
    # parse the HTML
    charset = 'utf-8'
    decoded_html = html_part.decode(charset)
    soup = BeautifulSoup(decoded_html, 'html.parser')
    jobs = soup.find(class_ = "scaffold-layout__list-container")
    job_info = jobs.find_all(class_ = "disabled ember-view job-card-container__link job-card-list__title")
    # extract the url for each job post
    for i in job_info:
        url = i['href']
        urls.append(url)
    return urls

In [411]:
job_urls = {}
i = 0
path = "D:/Study Abroad/course/DSCI441/project/webpages"

# get the urls of all job posts from the local job pages
for root, dirs, files in os.walk(path):
    for file in files:
        file_path = os.path.join(root, file)
        urls = job_url(file_path)
        for url in urls:
            job_urls[i] = {}
            job_urls[i]["url"] = url
            job_urls[i]["status"] = 0
            i += 1

(3). Download Job Post Details Webpages

In [40]:
# manually login the linkedin website
driver = webdriver.Chrome()
driver.get("https://www.linkedin.com/home")

# wait enough time to log in manually
time.sleep(20)  

# save the cookies to a file after login
pickle.dump(driver.get_cookies(), open("D:/Study Abroad/course/DSCI441/project/cookies.pkl", "wb"))

driver.quit()

In [14]:
# use selenium webdriver to iterate all the job pages
# set the options
options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# load the previously saved cookies
cookies = pickle.load(open("D:/Study Abroad/course/DSCI441/project/cookies.pkl", "rb"))

# iterate all job post pages and download them to local path
for i in range(0,1000):
    index = i
    value = job_urls[index]
    
    # open a new driver after iterating the job post pages for 20 times in order to save memory
    if index % 20 == 0:
        driver = webdriver.Chrome(options=options)
        driver.get("https://www.linkedin.com/home")
        time.sleep(4)
        
        # load the cookies to the driver
        for cookie in cookies:
            driver.add_cookie(cookie)
        time.sleep(1)
    
    # get the url for job post
    url = value["url"]
    driver.get(url)
    
    time.sleep(5)
    
    # use the aria-label to locate the button
    button_aria_label = "Click to see more description"
    try:
        # Wait up to 10 seconds for the button to be clickable
        button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, f"//button[@aria-label='{button_aria_label}']")))
        # click the button "Show More" to show the entire job description
        button.click()
        
        time.sleep(2) # wait up for all information displayed
        
    except Exception as e:
        break # if failed, jump to the next iteration
    
    pyautogui.hotkey('ctrl', 's') # save the page to local path
    time.sleep(2)
    
    file_name = "data_scientist_job_post_" + str(index)
    pyperclip.copy(file_name) # copy the file name
    
    pyautogui.hotkey('ctrl', 'v') # paste the file name
    time.sleep(2)
    
    if index % 20 == 0:
        pyautogui.hotkey('tab')  # select the save type
        time.sleep(1)

        pyautogui.hotkey('down')
        time.sleep(1)

        pyautogui.hotkey('up')  # save file to mhtml
        time.sleep(1)

        pyautogui.hotkey('enter') # confirm the file tyep
        time.sleep(1)
    
    pyautogui.hotkey('enter') # save
    time.sleep(np.random.randint(1,4)) # wait for random time
    
    if index % 20 == 19:
        driver.quit()

(4). Extract Key Information from Job Posting Webpages

In [119]:
# categorize all job_type, remote, and experience
JOB_TYPE = ["Full-time","Part-time","Contract","Temporary","Internship","Other"]
REMOTE_TYPE = ["On-site","Remote","Hybrid"]
EXPERIENCE_TYPE = ["Internship","Entry level","Associate","Mid-Senior level","Director","Executive"]

In [147]:
# define functions to extract job_type, remote, and experience information
def extract_info(info, options):
    for i in options:
        if i in info:
            return i
    return None

In [490]:
# create variables to store the details of all the jobs
indice = []
titles = []
companies = []
locations = []
job_types = []
remotes = []
experiences = []
salaries = []
company_sizes = []
fields = []
job_descriptions = []
skills_posted = []
skills_associated = []

In [491]:
# get the urls of all job posts from the local job pages
path = "D:/Study Abroad/course/DSCI441/project/jobposts"

for root, dirs, files in os.walk(path):
    for file in files:
        file_path = os.path.join(root, file)
        # Open the MHTML file in binary mode and parse it
        with open(file_path, 'rb') as file:
            msg = BytesParser(policy=policy.default).parse(file)

        # Decode the HTML part correctly
        html_part = None
        for part in msg.walk():
            content_type = part.get_content_type()
            if content_type == 'text/html':
                html_part = part.get_payload(decode=True)
                break

        # parse the HTML
        charset = 'utf-8'
        decoded_html = html_part.decode(charset)
        soup = BeautifulSoup(decoded_html, 'html.parser')
        
        # extract the index
        index = file_path.replace(".mhtml","").split("_")[-1]
        indice.append(index)
        
        # extract title
        title = soup.find(class_="t-24 t-bold job-details-jobs-unified-top-card__job-title").contents[0].strip()
        titles.append(title)
        
        # extract company and location
        company_location = soup.find(class_="job-details-jobs-unified-top-card__primary-description-without-tagline mb2").text.strip()
        company = company_location.split('·')[0].strip()
        location = company_location.split('·')[1].strip()
        companies.append(company)
        locations.append(location)
        
        # extract job_type, remote, experience, and salary
        salary_jobtype_Remote_Experience = soup.find(class_="job-details-jobs-unified-top-card__job-insight job-details-jobs-unified-top-card__job-insight--highlight").text
        job_type = extract_info(salary_jobtype_Remote_Experience,JOB_TYPE)
        remote = extract_info(salary_jobtype_Remote_Experience,REMOTE_TYPE)
        experience = extract_info(salary_jobtype_Remote_Experience,EXPERIENCE_TYPE)
        salary = salary_jobtype_Remote_Experience.strip().split('\n')[0]
        job_types.append(job_type)
        remotes.append(remote)
        experiences.append(experience)
        salaries.append(salary)
        
        # extract company size and field
        companysize_field = soup.find_all(class_ = "job-details-jobs-unified-top-card__job-insight")[1].text.split('·')
        company_size = companysize_field[0].strip()
        company_sizes.append(company_size)
        if len(companysize_field)>1: 
            field = companysize_field[1].strip() 
        else:
            field = None
        fields.append(field)
        
        # extract the job description
        job_description = soup.find(class_ = "jobs-box__html-content jobs-description-content__text t-14 t-normal jobs-description-content__text--stretch").text.strip()
        job_descriptions.append(job_description)

        # extract the associated skills and skills added by job posters
        skill_posted = [item.text for item in soup.find_all(class_="pt5")[1].find_all("a",class_="app-aware-link job-details-how-you-match__skills-item-subtitle t-14 overflow-hidden")]
        skill_associated = [item.text for item in soup.find_all(class_="pt5")[1].find_all("a",class_="app-aware-link job-details-how-you-match__skills-section-descriptive-skill t-14")]
        skills_posted.append(skill_posted)
        skills_associated.append(skill_associated)

In [511]:
df = pd.DataFrame((titles,companies,locations,job_types,remotes,experiences,salaries,company_sizes,fields,job_descriptions,skills_posted,skills_associated,indice)).T
df.columns=["Job_Title","Company_Name","Location","Job_Type","Remote","Experience_Level","salary","Company_Size","Field","Job_Description","Posted_Skills","Associated_Skills","Files"]
df

,Job_Title,Company_Name,Location,Job_Type,Remote,Experience_Level,salary,Company_Size,Field,Job_Description,Posted_Skills,Associated_Skills,Files
0,Data Scientist,Globant,"Orlando, FL",Contract,Remote,Mid-Senior level,$60/hr - $70/hr,"10,001+ employees",IT Services and IT Consulting,About the job\n \n \nHigh familiari...,[],[Business Development · Cold Calling · Custome...,0
1,Data Scientist,The Brite Group Incorporated,"Herndon, VA",Full-time,Hybrid,None,"$100,000/yr - $135,000/yr",51-200 employees,None,About the job\n \n \nSkills Require...,[],[Artificial Intelligence (AI) · Data Science ·...,1
2,Data Scientist,RVO Health,"Minneapolis, MN",Full-time,Hybrid,Entry level,"$94,400/yr - $120,000/yr","1,001-5,000 employees",Hospitals and Health Care,About the job\n \n \nAT A GLANCEWe ...,[],[Machine Learning · R (Programming Language) ·...,10
3,"Data Scientist, Mid",Booz Allen Hamilton,"Alexandria, VA",Full-time,None,None,"$93,300/yr - $212,000/yr","10,001+ employees",IT Services and IT Consulting,About the job\n \n \nJob Number: R0...,[],[Natural Language Processing (NLP) · Statistic...,100
4,"Data Scientist, SEO",Grammarly,"Virginia, United States",Full-time,Hybrid,Mid-Senior level,"$181,000/yr - $281,000/yr","1,001-5,000 employees",Software Development,About the job\n \n \nGrammarly is e...,[],[Problem Solving · Predictive Analytics · R (P...,101
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,"Data Scientist, Mid",Booz Allen Hamilton,"San Diego, CA",Full-time,None,None,"$73,100/yr - $166,000/yr","10,001+ employees",IT Services and IT Consulting,About the job\n \n \nJob Number: R0...,[],[Data Mining · Pattern Recognition · Data Clea...,995
996,Principal Data Scientist - BCG X & BCG Fed,BCG X,"Boston, MA",Full-time,None,Mid-Senior level,"$252,900/yr","1,001-5,000 employees",Business Consulting and Services,About the job\n \n \nWho We AreBost...,[],[Data Visualization · Pattern Recognition · St...,996
997,Senior Data Scientist,The Walt Disney Company,"Santa Monica, CA",Full-time,On-site,Mid-Senior level,"$136,038/yr - $182,490/yr","10,001+ employees",Entertainment Providers,About the job\n \n \nThe Walt Disne...,[],[Data Analytics · Analytics · Mathematics · Da...,997
998,Data Scientist | AI Products,Red Ventures,"Charlotte, NC",Full-time,Hybrid,Entry level,"$80,000/yr - $150,000/yr","1,001-5,000 employees","Technology, Information and Internet",About the job\n \n \nWe are looking...,[],[Machine Learning · Problem Solving · Python (...,998
